**Importación de bibliotecas necesarias**
<br>
Para realizar este proyecto se utilizan las siguientes bibliotecas: 
+ numpy para trabajar con arrays
+ math para el cáluculo de pendientes 
+ tkinker para poder capturar en una ventana la posición del ratón y dibujar los pixeles del recorrido del ratón.

In [1]:
import numpy as np
import math
from tkinter import *

## Algoritmo Forward
En el siguiente bloque de código tenemos 3 variaciones del algoritmo forward. Todos esperan como parámetros de entrada:
+ V: Array de secuencia de observaciones.
+ a: Matriz (cuadrada) de transición entre estados.
+ b: Matriz de probabilidad de emisión nxm, donde *n* es el número de estados y *m* es el número de posibles observaciones.
+ pi: Array de la distribución inicial.

1. Otorgamos a la Variable T el tamaño que tenga nuestra secuencia de observaciones. 
2. Creamos un array lleno de 0s de tamaño T por el tamaño de 'a' en uno de sus ejes.
3. En t =0 Se calculan los valores iniciales de $ \alpha $ multiplicando la distribución inicial por b
4. Iteramos desde $t=1$ hasta $t<T$ y para cada fila se recalcula $\alpha$ utilizando los valores de la columna anterior.
5. Se retorna $ \alpha $

A grandes rasgos podemos ver como funciona este algoritmo en el siguiente diagrama de trellis: <br>
![title](\assets\trellis.jpg)
<br> 
*Diagrama de Trellis para el Algoritmo Forward <a href="https://www.idiap.ch/~odobez/TEACHING/EE613/EE613-HMM.pdf" target="_blank">Créditos</a>*

*Forwardprob()*

Agrega las variables *p* con valor inicial de 1 y  *probability_of_observation* con valor inicial 0, la cual irá auto sumado los valores de $\alpha$ posteriormente estos se multiplican por *p* y retornamos el valor de *p*. Como posteriormente se puede observar estos valores son propensos a tener underflow y ser redondeados a 0.

*logForward()*

Para evitar los problemas de underflow antes mencionados, se crea una lista *scale* de tamaño T donde se guarda un factor de escalamiento para los valores de $\alpha$ posteriormente se hace una sumatoria de los logaritmos de dichos valores. Utilizando la función .sum() de numpy

In [2]:
def forward(V, a, b, pi):
    T = V.shape[0]
    alpha = np.zeros((T, a.shape[0]))
    alpha[0, :] = pi * b[:, V[0]]

    for t in range(1, T):
        for j in range(a.shape[0]):
            # Matrix Computation Steps
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]

    return alpha


def probforward(V, a, b, pi):
    """Forward returns prob, it may occur underflow"""
    p = 1
    alpha = np.zeros((V.shape[0], a.shape[0]))
    alpha[0, :] = pi * b[:, V[0]]

    for t in range(1, V.shape[0]):
        probability_of_observation = 0
        for j in range(a.shape[0]):
            alpha[t, j] = alpha[t - 1].dot(a[:, j]) * b[j, V[t]]
            probability_of_observation += alpha[t, j]
        p = p * probability_of_observation
    # print(alpha)
    return p

def logForward(V, a, b, pi):
    """Forward returns logprob"""
    T = V.shape[0]
    scale = np.zeros(T)
    alpha = np.zeros((T, a.shape[0]))

    alpha[0, :] = pi * b[:, V[0]]
    scale[0] = alpha[0, :].sum()
    # alpha scaled
    alpha[0] = alpha[0]/scale[0]
    # Induction steps
    for t in range(1, T):
        alpha_hat = alpha[t - 1].dot(a) * b[:, V[t]]
        # alpha scaling 
        scale[t] = alpha_hat.sum()
        alpha[t] = alpha_hat / scale[t]
    # print(alpha)
    return np.log(scale).sum()


## Algoritmo Backwards 

Backwards es básicamente la implementación en reversa del algoritmo Forward, a diferencia que se avanza desde T hasta 0 y no se emplean los valores de la distribución inicial sino los valores de $\Beta$ serán 1 solo cuando $t=T$ y cuando $t>T$ se calculan con la columna $t+1$

Se puede ver con mayor claridad con el siguiente diagrama de trellis: <br>
![title](\assets\trellisback.jpg)
<br> 
*Diagrama de Trellis para el Algoritmo Backwards <a href="https://www.idiap.ch/~odobez/TEACHING/EE613/EE613-HMM.pdf" target="_blank">Créditos</a>*


In [3]:
def backward(V, a, b):
    T = V.shape[0]
    beta = np.zeros((T, a.shape[0]))
    beta[T - 1] = np.ones((a.shape[0]))

    # Loop in backward way from T-1 to
    # Due to python indexing the actual loop will be T-2 to 0
    for t in range(T - 2, -1, -1):
        for j in range(a.shape[0]):
            beta[t, j] = (beta[t + 1] * b[:, V[t + 1]]).dot(a[j, :])
    # print(beta)
    return beta


## Algoritmo Baum Welch

Recibe como entrada los siguientes parámetros:

+ V: Array de secuencia de observaciones.
+ a: Matriz (cuadrada) de transición entre estados.
+ b: Matriz de probabilidad de emisión nxm, donde *n* es el número de estados y *m* es el número de posibles observaciones.
+ pi: Array de la distribución inicial.
+ n_iter (opcional): numero máximo de iteraciones.

1. Asignamos a la variables T y M el tamaño de la secuencia de observaciones y la transición entre estados respectivamente.
2. Iteramos
    1. calculamos $\alpha$ utilizando la función de forward.
    2. calculamos $\beta$ utilizando la función de forward.
    3. creamos el array $\xi$ de dimensiones $M*M*T$
    4. Iteramos hasta T-1
        1. calculamos el denominador de $\xi$ multiplicando $\alpha$ en el instante $t$ por la matriz a y la matriz b por la prob de emitir la observación en el instante $t+1$  
        1. Iteramos hasta M
            1. calculamos el numerador de $\xi$ multiplicando $\alpha$ en los índices $t, i$ por la matriz a y la matriz b por la prob de emitir la observación en el instante $t+1$  
    5. Se calculan los valores de $\gamma$ utilizando la función .sum de numpy en su eje 1 
    6. Actualizamos los valores de **a** sumando en $\gamma$ en su eje 2 y se divide entre la suma de $\gamma$ en el eje 1 
    7. Se suma $\xi$ en su eje 3 y se le agrega a $\gamma$
    8. iteramos a traves del eje 2 de **b**
        1. Se calculan los valores únicos de cada observación.
    8. Se actualiza el valor de **b**  al dividirlo entre la suma de $\gamma$ en su eje 1
9. Se retorna un diccionario con los valores de **a** y **b**


In [4]:
def baum_welch(V, a, b, pi, n_iter=100):
    T = V.shape[0]
    M = a.shape[0]

    for n in range(n_iter):
        alpha = forward(V, a, b, pi)

        # print(alpha)
        beta = backward(V, a, b)

        # xi array m*m*t
        xi = np.zeros((M, M, T - 1))
        for t in range(T - 1):
            denominator = np.dot(
                np.dot(alpha[t, :].T, a) * b[:, V[t + 1]].T, beta[t + 1, :])
            for i in range(M):
                numerator = alpha[t, i] * a[i, :] * \
                    b[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator

        gamma = np.sum(xi, axis=1)
        # print(gamma)
        a = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))

        # Add additional T element in gamma
        gamma = np.hstack(
            (gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))

        K = b.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            b[:, l] = np.sum(gamma[:, V == l], axis=1)

        b = np.divide(b, denominator.reshape((-1, 1)))

    return {"a": a, "b": b}

## Funciones útiles 
*stochastic()* <br>
Esta función recibe una matriz y la retorna en una matriz estocástica.

*stochastic()* <br>
Esta función recibe una matriz y un valor por el cual reemplazará los 0's que en ella se encuentran.


In [5]:
# Utils

def stochastic(matrix):
    """Receives  a matrix and returns it row stochastic, row sum equals 1"""
    return matrix/matrix.sum(axis=1)[:, None]


def smoothMatx(mat, smoothing=1e-5):
    """ sets floor value to 1e-5, smoothing value can be provided"""
    
    mat[mat == 0] = smoothing
    return mat


# Recreamos el experimento de la presentación de Dr. Sung-Jung Cho de HMM

El siguiente bloque a manera de prueba previa se evalúan el funcionamiento de los algoritmos al entrenar un modelo oculto de Markov con una secuencia de observaciones *V*, una matriz de transición *a* de $3*3$ con valores aleatorios que posteriormente será modificada para ser estocástica, una matriz de probabilidades de emisión *b* de $3*16$ y una distribución inicial *pi* de 3 estados. 



Al entrenar el modelo tenemos la siguiente matriz de transición:

```
 [0.66666667 0.33333333 0.        ]
 [0.         0.83333333 0.16666667]
 [0.         0.         1.        ]
```

la cual se puede interpretar con el siguiente diagrama: <br>
![title](\assets\ltr.jpg)
<br> 





In [6]:


# Preflight Test

# obs 0 demo
V = np.array([7, 7, 6, 5, 5, 4, 4, 3, 1, 0, 15, 15, 14, 13, 12, 12, 11, 10, 9])
# V = np.array([8, 8, 6, 5, 5, 4, 4, 3, 2, 1, 0, 1,
#              15, 14, 13, 12, 12, 11, 10, 9, 9])

# Transition Probabilities
a = np.random.rand(3, 3)
a[1, 0] = 0
a[2, 0] = 0
a[2, 1] = 0
a = stochastic(a)

# Emission Probabilities
b = np.random.rand(3, 16)
b = stochastic(b)
pi = np.array((1, 0, 0))

# train model with demo 0 samples
fit1 = baum_welch(V, a, b, pi, n_iter=100)
print(f'Trans Mat: \n {fit1["a"]}')
print(f'Obs Mat: \n {fit1["b"]}')


a = fit1["a"]
b = fit1["b"]

# B matrix smoothing, change 0's to 1e-5
bsmooth = smoothMatx(b) 

# Classification with demos of 0 and 1 

p0 = probforward(np.array([7, 6, 5, 4, 3, 3, 2, 1, 0, 15, 14,
               13, 12, 11, 10, 9, 9, 8]), a, bsmooth, pi)  
print(f"0 prob: {p0}")
p1 = probforward(np.array([4, 4, 4, 4, 4]), a, bsmooth, pi)  
print(f"1 prob: {p1}")


logp0 = logForward(np.array([7, 6, 5, 4, 3, 3, 2, 1, 0, 15, 14,
               13, 12, 11, 10, 9, 9, 8]), a, bsmooth, pi) 
print(f"0 Logprob: {logp0}")

logp1 = logForward(np.array([4, 4, 4, 4, 4]), a, bsmooth, pi)
print(f"1 Logprob: {logp1}")


Trans Mat: 
 [[0.8 0.2 0. ]
 [0.  0.8 0.2]
 [0.  0.  1. ]]
Obs Mat: 
 [[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  4.48283971e-45 4.00000000e-01 2.00000000e-01 4.00000000e-01
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.00000000e-01 2.00000000e-01 0.00000000e+00 2.00000000e-01
  4.00000000e-01 8.80761388e-26 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.09879784e-22]
 [1.68945048e-15 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.11111111e-01 1.11111111e-01 1.11111111e-01
  2.22222222e-01 1.11111111e-01 1.11111111e-01 2.22222222e-01]]
0 prob: 2.012062425307512e-193
1 prob: 1.776154757838997e-185
0 Logprob: -54.794295629370964
1 Logprob: -108.06009771836526


## Arreglos de observaciones
El siguiente bloque contiene 3 lista para el entrenamiento de cada dígito. 

In [7]:

# Obs array to train models 
# 0
dig0 = [8, 8, 6, 5, 5, 4, 4, 3, 2, 1, 0, 0, 15, 14, 13, 12, 12, 11, 10, 9, 9]
dig00 = [8, 7, 7, 5, 4, 3, 1, 0, 14, 14, 13, 12, 12, 11]
dig000 = [8, 7, 6, 5, 4, 4, 3, 3, 2, 1, 0, 0, 0, 13, 11, 11, 11, 10, 10, 9]
# 1
dig1 = [4, 4, 4, 4, 4, 4, 4, 4]
dig01 = [4, 4, 4, 4, 4, 4, 3]
dig001 = [4, 4, 4, 4, 4, 4, 4, 4, 5]
# 2
dig2 = [14, 14, 15, 2, 5, 6, 6, 5, 5, 2, 0, 0, 0]
dig02 = [13, 14, 15, 0, 2, 4, 5, 6, 6, 6, 6, 6, 5, 1, 0, 0, 0, 0]
dig002 = [13, 14, 0, 2, 3, 4, 5, 6, 6, 6, 6, 1, 1, 1]
# 3
dig3 = [0, 0, 0, 4, 6, 7, 7, 0, 0, 2, 4, 6, 7, 8]
dig03 = [0, 0, 0, 0, 7, 7, 7, 7, 1, 0, 1, 2, 5, 6, 7, 8, 8]
dig003 = [0, 0, 1, 4, 6, 7, 8, 0, 0, 0, 1, 3, 5, 6, 7, 8, 8]
# 4
dig4 = [8, 8, 8, 8, 8, 8, 15, 15, 15, 14, 14, 14, 14, 14, 5, 4, 4, 4, 4, 4, 4, 4]
dig04 = [8, 8, 8, 8, 8, 8, 15, 15, 15, 14, 14, 14, 14, 2, 4, 4, 4, 4, 4]
dig004 = [7, 8, 8, 8, 8, 15, 0, 15, 13, 13, 13, 13, 1, 4, 4, 4, 4, 4]
# 5
dig5 = [8, 8, 8, 8, 8, 7, 3, 4, 0, 0, 0, 1, 2, 2, 3, 4, 6, 7, 8, 9, 9]
dig05 = [8, 8, 8, 7, 4, 3, 0, 0, 0, 2, 2, 3, 4, 6, 7, 8, 8]
dig005 = [8, 8, 8, 7, 4, 4, 0, 0, 0, 1, 1, 3, 4, 5, 7, 8, 9, 10, 11]
# 6
dig6 = [7, 6, 6, 5, 4, 3, 1, 0, 15, 13, 12, 9, 7, 7, 6]
dig06 = [8, 7, 6, 5, 4, 3, 3, 2, 1, 1, 0, 14, 12, 11, 9, 8, 7, 6, 5]
dig006 = [7, 7, 6, 5, 4, 4, 3, 2, 1, 0, 14, 12, 9, 7]
# 7
dig7 = [0, 0, 0, 0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6]
dig07 = [0, 0, 0, 0, 0, 0, 2, 5, 5, 5, 5, 4, 5, 5, 5, 5]
dig007 = [0, 0, 0, 0, 0, 2, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5]
# 8
dig8 = [8, 7, 4, 1, 0, 1, 2, 3, 5, 8, 9, 10, 11, 12, 14, 14, 14, 10]
dig08 = [10, 8, 7, 6, 4, 3, 0, 0, 0, 1, 2, 3, 5,
         6, 8, 8, 9, 9, 11, 12, 13, 14, 15, 15, 14, 13]
dig008 = [9, 8, 8, 5, 4, 3, 1, 1, 1, 1, 2, 3, 3, 4, 6, 8,
          8, 9, 10, 11, 12, 13, 14, 15, 15, 15, 13, 12, 10, 9]
# 9
dig9 = [9, 7, 5, 4, 1, 0, 12, 12, 4, 4, 4, 4, 4, 4]
dig09 = [8, 7, 5, 4, 2, 0, 14, 12, 12, 4, 4, 4, 4, 4, 4, 4]
dig009 = [8, 8, 6, 3, 1, 15, 12, 13, 4, 4, 3, 3, 4, 4, 4]


## Funciones de entrenamiento y evaluación

Con el fin de tener un código más limpio se escribieron las siguientes funciones.

*fitnum()* <br>

Se encarga de entrenar un modelo oculto de Markov.

Entrada: 
+ dig: array del dig a entrenar
+ n: número de estados ocultos
+ m: número de símbolos que se pueden emitir.
<br>
Salida: 
+ fit : diccionario con la matrices *a* y *b* optimizadas por el algoritmo de Baum Welch
+ pi: distribución inicial


*evalNum()* <br>
Se encarga del problema de la evaluación de un modelo oculto de Markov.
Entrada: 
+ obs: array del dig a entrenar
+ a: matriz de transición 
+ b: matriz de emisión de observaciones. 
<br>
Salida: 
+ logForward : log probabilidad que los datos hayan sido generados dados los parámetros del modelo.



In [8]:
def fitnum(dig, n, m):
    #transition probabilities
    a = np.random.rand(n, n)
    a = np.triu(a)
    a = stochastic(a)

    # emission probabilities
    b = np.random.rand(n, m)
    b = stochastic(b)

    # initial distribution
    pi = np.zeros(n)
    pi[0] = 1

    # train model
    fit = baum_welch(np.array(dig),a, b, pi)

    return fit, pi 


def evalNum(obs, a, b, pi):
    b = smoothMatx(b)
    return logForward(np.array(obs), a, b, pi)


## Entrenamiento y Evaluación 

La primer parte del siguiente bloque entrena los modelos de cada dígito. 

*eval()*<br>
Entrada:<br>
Obs: Secuencia de observaciones. <br>
Salida: <br>
maxlogprob: Máxima log probabilidad.

1.  Se crea el array *logprob* vacío
2.  Para cada dígito y secuencia de entrenamiento se ajusta un modelo
1.  A cada modelo se le pasa la secuencia de observaciones a evaluar.
1.  Utilizando la función .max() de numpy se guarda la que tenga mayor log probabilidad de cada dígito.
1.  Se empuja la anterior log prob al array *logprob*
1.  Utilizando la función .max() de numpy obtenemos el mayor de todos los modelos.
1. Se imprime el índice y su valor en consola.




In [9]:
# Training phase 


ALLSYMBOLS = 16

# <==== Training for 0 ====>
fit0, pi0 = fitnum(dig0, 3, ALLSYMBOLS)
fit00, pi00 = fitnum(dig00, 3, ALLSYMBOLS)
fit000, pi000 = fitnum(dig000, 3, ALLSYMBOLS)


# # <==== Training for 1 ====>
fit1, pi1 = fitnum(dig1, 2, ALLSYMBOLS)
fit01, pi01 = fitnum(dig01, 2, ALLSYMBOLS)
fit001, pi001 = fitnum(dig001, 2, ALLSYMBOLS)
# fit01 = baum_welch(V, a, b, pi)
# fit001 = baum_welch(V, a, b, pi)


# <==== Training for 2 ====>
fit2, pi2 = fitnum(dig2, 3, ALLSYMBOLS)
fit02, pi02 = fitnum(dig02, 3, ALLSYMBOLS)
fit002, pi002 = fitnum(dig002, 3, ALLSYMBOLS)


# <==== Training for 3 ====>
fit3, pi3 = fitnum(dig3, 4, ALLSYMBOLS)
fit03, pi03 = fitnum(dig03, 4, ALLSYMBOLS)
fit003, pi003 = fitnum(dig003, 4, ALLSYMBOLS)


# <==== Training for 4 ====>
fit4, pi4 = fitnum(dig4, 3, ALLSYMBOLS)
fit04, pi04 = fitnum(dig04, 3, ALLSYMBOLS)
fit004, pi004 = fitnum(dig004, 3, ALLSYMBOLS)


# <==== Training for 5 ====>
fit5, pi5 = fitnum(dig5, 4, ALLSYMBOLS)
fit05, pi05 = fitnum(dig05, 4, ALLSYMBOLS)
fit005, pi005 = fitnum(dig005, 4, ALLSYMBOLS)


# <==== Training for 6 ====>
fit6, pi6 = fitnum(dig6, 4, ALLSYMBOLS)
fit06, pi06 = fitnum(dig06, 4, ALLSYMBOLS)
fit006, pi006 = fitnum(dig006, 4, ALLSYMBOLS)


# <==== Training for 7 ====>
fit7, pi7 = fitnum(dig7, 2, ALLSYMBOLS)
fit07, pi07 = fitnum(dig07, 2, ALLSYMBOLS)
fit007, pi007 = fitnum(dig007, 2, ALLSYMBOLS)


# <==== Training for 8 ====>
fit8, pi8 = fitnum(dig8, 4, ALLSYMBOLS)
fit08, pi08 = fitnum(dig08, 4, ALLSYMBOLS)
fit008, pi008 = fitnum(dig008, 4, ALLSYMBOLS)

# <==== Training for 9 ====>
fit9, pi9 = fitnum(dig9, 4, ALLSYMBOLS)
fit09, pi09 = fitnum(dig09, 4, ALLSYMBOLS)
fit009, pi009 = fitnum(dig009, 4, ALLSYMBOLS)



# evaluation phase with the observation sequence.
def eval(obs):

    logprob = []

    logp0 = evalNum(np.array(obs), fit0["a"], fit0["b"], pi0)
    logp00 = evalNum(np.array(obs), fit00["a"], fit00["b"], pi0)
    logp000 = evalNum(np.array(obs), fit000["a"], fit000["b"], pi00)
    log0 = max([logp0, logp00, logp000])
    print(f"0 Logprob: {log0}")
    logprob.append(log0)

    logp1 = evalNum(np.array(obs), fit1["a"], fit1["b"], pi1)
    logp01 = evalNum(np.array(obs), fit01["a"], fit01["b"], pi01)
    logp001 = evalNum(np.array(obs), fit001["a"], fit001["b"], pi001)
    log1 = max([logp1, logp01, logp001])
    print(f"1 Logprob:{log1}")
    logprob.append(log1)

    logp2 = evalNum(np.array(obs), fit2["a"], fit2["b"], pi2)
    logp02 = evalNum(np.array(obs), fit02["a"], fit02["b"], pi02)
    logp002 = evalNum(np.array(obs), fit002["a"], fit002["b"], pi002)
    log2 = max([logp2, logp02, logp002])
    print(f"2 Logprob:{log2}")
    logprob.append(log2)

    logp3 = evalNum(np.array(obs), fit3["a"], fit3["b"], pi3)
    logp03 = evalNum(np.array(obs), fit03["a"], fit03["b"], pi03)
    logp003 = evalNum(np.array(obs), fit003["a"], fit003["b"], pi003)
    log3 = max([logp3, logp03, logp003])
    print(f"3 Logprob:{log3}")
    logprob.append(log3)

    logp4 = evalNum(np.array(obs), fit4["a"], fit4["b"], pi4)
    logp04 = evalNum(np.array(obs), fit04["a"], fit04["b"], pi04)
    logp004 = evalNum(np.array(obs), fit004["a"], fit004["b"], pi004)
    log4 = max([logp4, logp04, logp004])
    print(f"4 Logprob:{log4}")
    logprob.append(log4)

    logp5 = evalNum(np.array(obs), fit5["a"], fit5["b"], pi5)
    logp05 = evalNum(np.array(obs), fit05["a"], fit05["b"], pi05)
    logp005 = evalNum(np.array(obs), fit005["a"], fit005["b"], pi005)
    log5 = max([logp5, logp05, logp005])
    print(f"5 Logprob:{log5}")
    logprob.append(log5)

    logp6 = evalNum(np.array(obs), fit6["a"], fit6["b"], pi6)
    logp06 = evalNum(np.array(obs), fit06["a"], fit06["b"], pi06)
    logp006 = evalNum(np.array(obs), fit006["a"], fit006["b"], pi006)
    log6 = max([logp6, logp06, logp006])
    print(f"6 Logprob:{log6}")
    logprob.append(log6)

    logp7 = evalNum(np.array(obs), fit7["a"], fit7["b"], pi7)
    logp07 = evalNum(np.array(obs), fit07["a"], fit07["b"], pi07)
    logp007 = evalNum(np.array(obs), fit007["a"], fit007["b"], pi007)
    log7 = max([logp7, logp07, logp007])
    print(f"7 Logprob:{log7}")
    logprob.append(log7)

    logp8 = evalNum(np.array(obs), fit8["a"], fit8["b"], pi8)
    logp08 = evalNum(np.array(obs), fit08["a"], fit08["b"], pi08)
    logp008 = evalNum(np.array(obs), fit008["a"], fit008["b"], pi008)
    log8 = max([logp8, logp08, logp008])
    print(f"8 Logprob:{log8}")
    logprob.append(log8)

    logp9 = evalNum(np.array(obs), fit9["a"], fit9["b"], pi9)
    logp09 = evalNum(np.array(obs), fit09["a"], fit09["b"], pi09)
    logp009 = evalNum(np.array(obs), fit009["a"], fit009["b"], pi009)
    log9 = max([logp9, logp09, logp009])
    print(f"9 Logprob:{log9}")
    logprob.append(log9)

    maxlogprob = max(logprob)
    print(
        f"Number: {logprob.index(maxlogprob)} with logprob of:  {maxlogprob}")


### Ventana que captura los trazos y emite las observaciones.

El siguiente bloque se encarga de capturar el clic sostenido del ratón, el movimiento del mismo para generar las observaciones y por último capturar cuando el usuario ha soltado el botón del ratón para evaluar los modelos previamente entrenados y de esa forma poder reconocer el dígito trazado.

+ Con ayuda del Canvas se crea una ventana
+ La función *get_x_and_y()* obtienen y actualizan la coordenadas del puntero y reciben el evento del clic primario del ratón.
+ La función *draw()* se ejecuta mientras se mantiene presionado el botón primario del ratón y crea una linea en el recorrido que se ha tenido y guarda las coordenadas de *X* y *Y* en sus respectivos arreglos.
+ La función *release()* espera a que el usuario suelte el botón del ratón y pasa los arreglos de coordenadas a *genObser()*
<br>

*genObser()* <br>
Entrada: 
+ xlist: lista con las coordenadas X
+ ylist: lista con las coordenadas Y
<br>

1. Se crean las listas vacías de *xpurge, ypurge, obs*
1. Se itera sobre la lista *xlist* con incrementos de 20
    1. Se empuja el valor del indice actual a *xpurge*
    1. Se empuja el valor del indice actual a *ypurge*
1. se itera sobre la lista (xpurge)-1):
    1. calcular dy = ypurge[i]-ypurge[i+1]
    1. calcular dx = xpurge[i]-xpurge[i+1]
    1. calcular y convertir el ángulo
    1. Se agrega el angulo a la lista mydegrees
1. dependiendo el valor del ángulo se empuja la observación con valores del 0 al 15 al arreglo obs
1. Se imprime la secuencia de observaciones vía consola
1. Llamar función eval y pasar como parámetro el array obs.

![title](\assets\angulo.jpg) <br>
*Hay que resaltar que el valor del ángulo es en sentido anti-horario, debido a que los valores de Y están invertidos* 
 ```
 x,y  -------------
      |  1,1
      |    \
      |     \
      |     2,2
``` 

<br> 



In [14]:


app = Tk()
app.geometry("400x400")


canvas = Canvas(app, bg='white')
canvas.pack(anchor='nw', fill='both', expand=1)

xcords = []
ycords = []


def get_x_and_y(event):
    global lasx, lasy
    lasx, lasy = event.x, event.y


def draw(event):
    global lasx, lasy
    xcords.append(lasx)
    ycords.append(lasy)
    canvas.create_line((lasx, lasy, event.x, event.y),
                       fill='red',
                       width=2)
    lasx, lasy = event.x, event.y


def release(event):
    print("Click release!")
    genObser(xcords, ycords)


def genObser(xlist, ylist):
    """Generate observation sequence"""
    xpurge, ypurge = [], []
    deglist = []
    obs = []
    for i in range(0, len(xlist), 20):
        xpurge.append(xlist[i])
        ypurge.append(ylist[i])

    for i in range(len(xpurge)-1):
        dy = ypurge[i]-ypurge[i+1]
        dx = xpurge[i]-xpurge[i+1]
        myradians = math.atan2(dy, dx)
        mydegrees = math.degrees(myradians)
        deglist.append(mydegrees)
        # print(mydegrees)
    # Generate observation from 0-15 based on the angle 2 points
    for i in deglist:
        # 0
        if i >= 168.75 and i <= 180:
            obs.append(0)
        if i <= -168.75:
            obs.append(0)
        # 1
        if i > -168.75 and i < -146.25:
            obs.append(1)
        # 2
        if i > -146.25 and i < -123.75:
            obs.append(2)
        # 3
        if i > -123.75 and i < -101.25:
            obs.append(3)
        # 4
        if i > -101.25 and i < -78.75:
            obs.append(4)
        # 5
        if i > -78.75 and i < -56.25:
            obs.append(5)
        # 6
        if i > -56.25 and i < -33.75:
            obs.append(6)
        # 7
        if i > -33.75 and i < -11.25:
            obs.append(7)
        # 8
        if i >= 0 and i < 11.25:
            obs.append(8)
        if i < 0 and i > -11.25:
            obs.append(8)
        # 9
        if i >= 11.25 and i < 33.75:
            obs.append(9)
        # 10
        if i >= 33.75 and i < 56.25:
            obs.append(10)
        # 11
        if i >= 56.25 and i < 78.75:
            obs.append(11)
        # 12
        if i >= 78.75 and i < 101.25:
            obs.append(12)
        # 13
        if i >= 101.25 and i < 123.75:
            obs.append(13)
        # 14
        if i >= 123.75 and i < 146.25:
            obs.append(14)
        # 15
        if i >= 146.25 and i < 168.75:
            obs.append(15)
    print(f'Obs Sequence: {obs}')
    eval(np.array(obs))


canvas.bind("<Button-1>", get_x_and_y)
canvas.bind("<B1-Motion>", draw)
canvas.bind("<ButtonRelease-1>", release)

app.mainloop()


Click release!
Obs Sequence: [9, 7, 5, 4, 1, 15, 13, 13, 4, 4, 4, 4, 4, 4, 4]
0 Logprob: -63.372137991253624
1 Logprob:-73.52085886471761
2 Logprob:-103.14608624289932
3 Logprob:-83.45407686188068
4 Logprob:-57.21834252314608
5 Logprob:-84.77885599114543
6 Logprob:-72.9975409393232
7 Logprob:-88.10007075164056
8 Logprob:-67.05243741838987
9 Logprob:-40.552552657653514
Number: 9 with logprob of:  -40.552552657653514
